In [ ]:
from Common_Functions.CommonFunctions import *
from Patches_Functions.ExtractPatches import extract_patches
from sklearn.neighbors import NearestNeighbors
from color_transfer import color_transfer
# from irls import irls
# from segmentation.FaceDetectionSegmentation import segement
import cv2
from skimage.transform import pyramid_gaussian


In [ ]:
def irls(X,z,r,Iirls,patch_size,subsampling_gap):
    Xp=extract_patches(X,patch_size,subsampling_gap)
    num_of_patches=(np.shape(Xp)[0])*(np.shape(Xp)[1])
    w=(np.ones((num_of_patches,1)))
    # z should be initialized with patch matching
    print(np.shape(Xp)[0], np.shape(Xp)[1])
    print(np.shape(z), np.shape(Xp))
    for i in range(0, Iirls):
        for x in range(0, np.shape(Xp)[0]):
            for y in range(0,np.shape(Xp)[1]):
                e=Xp[x,y,0,:,:,:]-z[x*np.shape(Xp)[1]+y]
                e2=np.sum(e**2)**0.5+0.00001
                w[x*np.shape(Xp)[1]+y]=(e2**(r-2))
                Xp[x,y,0,:,:,:]+=(z[x*np.shape(Xp)[1]+y]-Xp[x,y,0,:,:,:])*w[x*np.shape(Xp)[1]+y]


def segement(image):


    # initialize the mask
    mask = np.zeros(image.shape[:2], np.uint8)

    # not sure what these are for in the algorithm
    backgroundModel = np.zeros((1, 65), np.float64) 
    foregroundModel = np.zeros((1, 65), np.float64) 


    img_height=image.shape[0]
    img_width= image.shape[1]
   

   # initialize a rectangle for the grabcut algorithm
    rectangle = (1,1,img_width,img_height)

    # run the grabcut algorithm for 8 iterations
    cv2.grabCut(image, mask, rectangle, backgroundModel, foregroundModel, 10, cv2.GC_INIT_WITH_RECT) 
   

    # generate the forground mask
    mask_forground = np.where((mask == 2)|(mask == 0), 0, 1).astype('uint8') 

    # get the final image
    result_img = image * mask_forground[:, :, np.newaxis] 

 
    result_mask = np.where((result_img>0),1,0).astype(float)

    # print(result_mask)

    return result_mask


def color_transfer(content,style):
    content=cv2.cvtColor(content, cv2.COLOR_BGR2LAB)
    style=cv2.cvtColor(style, cv2.COLOR_BGR2LAB)    
    content[:,:,0]-=np.mean(content[:,:,0])
    content[:,:,1]-=np.mean(content[:,:,1])
    content[:,:,2]-=np.mean(content[:,:,2])
    content[:,:,0]*=(np.std(style[:,:,0])/(np.std(content[:,:,0])+0.00001))
    content[:,:,1]*=(np.std(style[:,:,1]/(np.std(content[:,:,1])+0.00001)))
    content[:,:,2]*=(np.std(style[:,:,2]/(np.std(content[:,:,2])+0.00001)))
    content[:,:,0]+=np.mean(style[:,:,0])
    content[:,:,1]+=np.mean(style[:,:,1])
    content[:,:,2]+=np.mean(style[:,:,2])
    return cv2.cvtColor(content.astype(np.float32), cv2.COLOR_LAB2BGR)


def style_transfer(content, style, r, L, Iirls, patch_sizes, subsampling_gaps, Ialg, seg_mask):
    content = color_transfer(content,style)
    # content = np.pad(content, ((patch_sizes[0], patch_sizes[0]), (patch_sizes[0], patch_sizes[0]), (0, 0)))
    # seg_mask = np.pad(seg_mask, ((patch_sizes[0], patch_sizes[0]), (patch_sizes[0], patch_sizes[0]), (0, 0)))
    content_pyramid_tuple = tuple(pyramid_gaussian(content, channel_axis=-1, max_layer=L, downscale=1.5))
    style_pyramid_tuple = tuple(pyramid_gaussian(style, channel_axis=-1, max_layer=L, downscale=1.5))
    w_pyramid_tuple = tuple(pyramid_gaussian(seg_mask, channel_axis=-1, max_layer=L, downscale=1.5))
    
    content_pyramid = []
    style_pyramid = []
    w_pyramid = []  
    for i in range (0,L):
        content_pyramid.append(content_pyramid_tuple[i])
        style_pyramid.append(style_pyramid_tuple[i])
        w_pyramid.append(w_pyramid_tuple[i])
    X = content_pyramid[L-1]
    # X= random_noise(content_pyramid[L-1], mode="gaussian",mean=0,var=50)
    for l in range (L-1,-1,-1):
        X = np.pad(X, ((0, patch_sizes[0]), (0, patch_sizes[0]), (0, 0)), mode='constant', constant_values=(1,1))
        if(l == L-1):
            X= random_noise(X, mode="gaussian",mean=0,var=50)
        else:
            X= random_noise(X, mode="gaussian",mean=0,var=1)
        content_pyramid[l] = np.pad(content_pyramid[l], ((0, patch_sizes[0]), (0, patch_sizes[0]), (0,0)), mode='constant', constant_values=(1,1))
        w_pyramid[l] = np.pad(w_pyramid[l], ((0, patch_sizes[0]), (0, patch_sizes[0]), (0, 0)), mode='constant', constant_values=(1,1))
        for s in range(0,len(patch_sizes)):
            style_patches =  extract_patches(style_pyramid[l],(patch_sizes[s], patch_sizes[s], 3),subsampling_gaps[s])
            flatten_style_patches = style_patches.reshape(-1, patch_sizes[s] * patch_sizes[s] * 3)
            nn_model = NearestNeighbors(n_neighbors=1).fit(flatten_style_patches)    
            z=[]
            for k in range(0,Ialg):
                Xp=extract_patches(X,(patch_sizes[s], patch_sizes[s], 3),subsampling_gaps[s])
                flatten_Xp = Xp.reshape(-1, patch_sizes[s] * patch_sizes[s] * 3)
                for Xpatch in flatten_Xp:
                   Xpatch = [Xpatch]     
                   flatten_neighbour_patch = flatten_style_patches[nn_model.kneighbors(Xpatch)[1][0][0]]
                   z.append(flatten_neighbour_patch.reshape(patch_sizes[s], patch_sizes[s], 3))
                irls(X,z,r,Iirls,(patch_sizes[s],patch_sizes[s],3),subsampling_gaps[s]) 
                X=color_transfer(X.astype(np.float32),style_pyramid[l].astype(np.float32))
                X =((1.0-w_pyramid[l])* X).astype(np.float32) + (w_pyramid[l].astype(np.float32)*content_pyramid[l]).astype(np.float32)
                show_images([X])
        if (l>0) : 
            X = cv2.resize(X, (np.shape(content_pyramid[l-1])[1], np.shape(content_pyramid[l-1])[0]))       
        #remove padding
        X = X[0:-patch_sizes[0], 0:-patch_sizes[0], :]









style=io.imread("imgs/starry.jpg").astype(np.float32)/255
X=io.imread("imgs/vat.jpg").astype(np.float32)/255
# X = color_transfer(X,style)

con = io.imread("imgs/vat.jpg")
seg_mask = segement(con)

# X= random_noise(X, mode="gaussian",mean=0,var=0.01)
# print(X.shape, style.shape)
# shape = 40
# Xp=extract_patches(X,(shape,shape,3),shape)
# style_patches =  extract_patches(style,(shape,shape,3),shape)
# z=[]
# style_patches = style_patches.reshape(-1, shape * shape * 3)
# nn_model = NearestNeighbors(n_neighbors=1).fit(style_patches)
# 
# print(X.shape)
# Xp = Xp.reshape(-1, shape * shape * 3)
# 
# print(Xp[0], style_patches[0])
# 
# print(4, (Xp.shape))
# for Xpatch in Xp:
# 
#     temp_arr = [Xpatch]
#     index = nn_model.kneighbors(temp_arr)[1][0][0] 
#     print(index, nn_model.kneighbors(temp_arr)[0][0][0])
#     paa = style_patches[index]
#     z.append(style_patches[index])
#     #now do un reshape to be 20*20*3
#     new_patch = paa.reshape(shape,shape,3)
#     show_images([new_patch, Xpatch.reshape(shape,shape,3)])

    # X = np.array(z) # should be replaced by the new xpatches generated from irls

# show_images([X])
#content,style,r,L,Iirls,patch_sizes,subsampling_gaps, Ialg
style_transfer(X,style,0.8,5,10,[33, 21, 13,9],[28, 18, 8,5],3, seg_mask)

In [ ]:
style=io.imread("imgs/starry.jpg").astype(np.float32)/255

X=np.ones(np.shape(style)).astype(np.float32)
X = color_transfer(X,style)
con = io.imread("imgs/starry.jpg")
seg_mask = segement(con)
# X=random_noise(content, mode="gaussian",mean=0,var=0.002)
# show_images([X])
style_transfer(X,style,0.8,1,3,[40],[40],2,con)
# show_images([style,X])